In [1]:
import serial
from serial.tools import list_ports
import sys
from collections import namedtuple

from timeit import default_timer as timer
from time import sleep

In [10]:
def find_teensy_port(use_first=False, teensy_vid='16C0', teensy_pid='0483', teensy_ser=None):
    stringify = lambda xid: f'{xid:04x}'.upper() if xid else None
    
    ports = [p for p in list_ports.comports()]
    
    if teensy_vid is not None:
        ports = [p for p in ports if stringify(p.vid) == teensy_vid]
    
    if teensy_pid is not None:
        ports = [p for p in ports if stringify(p.pid) == teensy_pid]
    
    if len(ports) > 1 and not use_first:
        print('Too many devices found! (n={})'.format(len(ports)), file=sys.stderr)
        return
    
    return ports[0]
find_teensy_port().device

'COM5'

In [63]:
ser = serial.Serial(find_teensy_port().device, baudrate=115200)

In [79]:
ser.write(b'1')

start = timer()
ser.write(b'S')
print(ser.readline())
end = timer()

print((end-start)*1000, 'ms')

ser.write(b'0')

b'!\r\n'
0.7741376648482401 ms


1

In [65]:
for n in range(10):
    ser.write(b'S')
    ser.flush()
    ser.readline()
    ser.write(b'R')
    print(ser.readline().strip())
    sleep(.2)
    ser.reset_output_buffer()
    ser.reset_input_buffer()

b'525'
b'189'
b'196'
b'659'
b'341'
b'353'
b'419'
b'536'
b'1889'
b'490'


In [28]:
%timeit ser.write(b'1')

58.1 µs ± 736 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [80]:
ser.close()